In [4]:
# importar bibliotecas

import PyPDF2
import re
import sys
sys.path.insert(0, 'C:\\Users\\MatheusPereira\\OneDrive - Pontte\\Área de Trabalho\\automacaoRegistroCCI\\Scripts')
from lerContrato import lerContrato
import lerContrato as lc

In [5]:
contratoPath = r'C:\Users\MatheusPereira\OneDrive - Pontte\Área de Trabalho\automacaoRegistroCCI\Contratos\FI_Contrato_Cristiano_Assinatura Digital-Manifesto.pdf'

In [6]:
contrato = lerContrato(contratoPath)
quantidade = lc.numParticipantes(contratoPath)
contrato

{'valorTotal': '498038.00',
 'tabela': 'SAC',
 'Taxa ao Mes': 0.008,
 'registro': 42638.0,
 'valorLiquido': '440000.00',
 'prazoMes': '360',
 'valorPrimeiraParcelaComEncargos': '5542.62',
 'valorImóvel': '631000.00',
 'prazoContrato': '360',
 'ultimaParcela': '18/08/2052',
 'dataContrato': '',
 'valorPrimeiraParcela': '5369.12',
 'primeiraParcela': '8/09/2022',
 'indice': 'IPCA',
 'Matrícula': 'ULARDECOMPRAEVENDADEIMÓVELFINANCIAMENTOCOMPACTOADJETODEALIENAÇÃOFIDUCIÁRIAEMGARANTIAEMISSÃODECÉDULADECRÉDITOIMOBILIÁRIO–CCIEOUTRASAVENÇASNº0001537769/CMAEstapáginaéparteintegranteeinseparáveldoInstrumentoParticulardeCompraeVendadeImóvelFinanciamentocomPactoAdjetodeAlienaçãoFiduciáriaemGarantiaEmissãodeCéduladeCréditoImobiliário–CCIeOutrasAvençasnº0001537769/CMAPágina1CAMPO1–CREDORARAZÃOSOCIAL:QISOCIEDADEDECRÉDITODIRETOS.A.ENDEREÇOCOMERCIAL:AvenidaBrigadeiroFariaLimanº2.3911ºandarCOMPLEMENTO:Conjunto12BAIRRO:JardimPaulista',
 'Cartório': 'NTO PARTICULAR DE COMPRA E VENDA DE IMÓVEL, FINANCIAMENTO 

In [15]:
#Faz a leitura usando a biblioteca

pdf_file = open(contratoPath, 'rb')
read_pdf = PyPDF2.PdfFileReader(pdf_file)

#Extriar Texto Página 1 a 5
text=''
for i in range(0,9):
    #Ler Página PDF
    pageObj = read_pdf.getPage(i)
    #Extrair Texto
    text=text+pageObj.extractText()

#Tratar Texto (Remover Quebra de Linhas e espaços duplos)
text = re.sub('\r', '', text) 
text = re.sub('\n', '', text)
text = re.sub(' {2,}', ' ', text).strip(' ')


'CPF: 337.550.780-15 DATA DE NASCIMENTO: 18/06/1961 FILIAÇÃO: MOSE OLIBONI e IDALINA SCORTEGAGNA IDENTIDADE (tipo/número): RG: 5013209563 ÓRGÃO EMISSOR: SSP/RS NACIONALIDADE: BRASILEIRA PROFISSÃO: EMPRESÁRIA ESTADO CIVIL: CASADO com a Cônjuge Anuente desde 09 de janeiro de 1985 REGIME DE CASAMENTO (SE FOR CASADO): COMUNHÃO UNIVERSAL DE BENS, conforme registro nº 2.580, livro nº 03 do Oficial de Registro de Imóveis da Comarca de Flores da Cunha (RS) ENDEREÇO RESIDENCIAL E DOMICILIAR: AVENIDA RUI BARBOSA, 12 COMPLEMENTO: APTO 501 BAIRRO: SÃO FRANCISCO CIDADE: NITERÓI UF: RJ CEP: 24360-440 TELEFONE(S): (21) 99983 2620 EMAIL: alencaroliboni@terra.com.br NOME: LEONARDO COLODA OLIBONI CPF: 128.225.257-73 DATA DE NASCIMENTO: 05/03/1990 FILIAÇÃO: ALENCAR LUIZ OLIBONI e MARISTELA DE FÁTIMA COLODA OLIBONI IDENTIDADE (tipo/número): RG: 12.608.269-2 ÓRGÃO EMISSOR: DETRAN/RJ NACIONALIDADE: BRASILEIRA PROFISSÃO: ADMINISTRADOR ESTADO CIVIL: CASADO, desde 01 de junho de 2019, com Elisa Oberlaender Gar

In [34]:
if "HE_" in contratoPath:
    listaDePara = {'cpf':'CPF:','data de nascimento':'DATA DE NASCIMENTO:','endereço':'ENDEREÇO RESIDENCIAL E DOMICILIAR:','complemento':'COMPLEMENTO:','bairro':'BAIRRO:','cidade':'CIDADE:','uf':'UF:','cep':'CEP:',
                'telefone':'TELEFONE(S)','email':'EMAIL:'}

    #len(listaHE) # <--- Qtd de Itens na Lista
    listaKey = []
    listaValues = []
    for qtdParticipantes in range(0, quantidade):
        # Extraindo participantes da operação 
        começo = contrato[f'Participante{qtdParticipantes+1}']                        #inicio e fim da extração
        fim = 'CAMPO 3 -'
        #Pegar posição das variáveis auxiliares no texto
        inicioTopico = text.find(começo, 0)
        finalTopico = text.find(fim, 0)

        #Criar Paragráfo Auxiliar
        paragrafoAux = text[inicioTopico+len(começo)+1:finalTopico-1]
        paragrafoAux = re.sub('\s+',' ', paragrafoAux)
        paragrafoAux
        print(contrato[f'Participante{qtdParticipantes+1}'])
        for key, value in listaDePara.items():
            if key == 'endereço':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("COMPLEMENTO:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
            elif key == 'complemento':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("BAIRRO:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
            elif key == 'bairro':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("CIDADE:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
            elif key == 'cidade':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("UF:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
            elif key == 'telefone':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("EMAIL:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco].strip()
            else:
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find(" ", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]

            listaKey.append(f'{key}{qtdParticipantes+1}')
            listaValues.append(valorExtraido.strip())
elif "FI_" in contratoPath:
    listaDePara = {'cpf':'CPF:','data de nascimento':'DATA DE NASCIMENTO:','endereço':'ENDEREÇO RESIDENCIAL E DOMICILIAR:','complemento':'COMPLEMENTO:','bairro':'BAIRRO:','cidade':'CIDADE:','uf':'UF:','cep':'CEP:',
                'telefone':'TELEFONE(S)','email':'EMAIL:'}

    #len(listaHE) # <--- Qtd de Itens na Lista
    listaKey = []
    listaValues = []
    for qtdParticipantes in range(0, quantidade):
        # Extraindo participantes da operação 
        começo = contrato[f'Participante{qtdParticipantes+1}']                        #inicio e fim da extração
        fim = 'CAMPO 3 -'
        #Pegar posição das variáveis auxiliares no texto
        inicioTopico = text.find(começo, 0)
        finalTopico = text.find(fim, 0)

        #Criar Paragráfo Auxiliar
        paragrafoAux = text[inicioTopico+len(começo)+1:finalTopico-1]
        paragrafoAux = re.sub('\s+',' ', paragrafoAux)
        paragrafoAux
        print(contrato[f'Participante{qtdParticipantes+1}'])
#Criar Dicionario das duas Listas
dict_keyValue = dict(zip(listaKey,listaValues))
dict_keyValue

ALENCAR LUIZ OLIBONI
LEONARDO COLODA OLIBONI
MARINA COLODA OLIBONI


{'cpf1': '337.550.780-15',
 'data de nascimento1': '18/06/1961',
 'endereço1': 'AVENIDA RUI BARBOSA, 12',
 'complemento1': 'APTO 501',
 'bairro1': 'SÃO FRANCISCO',
 'cidade1': 'NITERÓI',
 'uf1': 'RJ',
 'cep1': '24360-440',
 'telefone1': '(21) 99983 2620',
 'email1': 'alencaroliboni@terra.com.br',
 'cpf2': '128.225.257-73',
 'data de nascimento2': '05/03/1990',
 'endereço2': 'RUA EURICO MANUEL DO CARMO, 02',
 'complemento2': 'BL. 2, AP 304',
 'bairro2': 'CHARITAS',
 'cidade2': 'NITERÓI',
 'uf2': 'RJ',
 'cep2': '24370090',
 'telefone2': '(21) 99983 2620',
 'email2': 'alencaroliboni@terra.com.br',
 'cpf3': '124.246.777-78',
 'data de nascimento3': '08/12/1988',
 'endereço3': 'RUA AMORA, 501',
 'complemento3': 'CASA G17',
 'bairro3': 'PIRATININGA',
 'cidade3': 'NITERÓI',
 'uf3': 'RJ',
 'cep3': '24350-590',
 'telefone3': '(21) 99983 2620',
 'email3': 'alencaroliboni@terra.com.br'}